In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json
import os
import pandas as pd

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Referer": "https://www.autocasion.com",
    "Accept-Language": "es-ES,es;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

ESTADO_FILE = "estado_extraccion.json"
CARPETA_BLOQUES = "bloques_detalles"
os.makedirs(CARPETA_BLOQUES, exist_ok=True)

TAMANO_BLOQUE = 1000


def cargar_estado():
    if os.path.exists(ESTADO_FILE):
        with open(ESTADO_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    else:
        return {}

def guardar_estado(estado):
    with open(ESTADO_FILE, "w", encoding="utf-8") as f:
        json.dump(estado, f, indent=2, ensure_ascii=False)

def extraer_detalles(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        ul_ficha = soup.find("ul", class_="datos-basicos-ficha")
        detalles = [li.get_text(strip=True) for li in ul_ficha.find_all("li")] if ul_ficha else []
        return detalles
    except Exception as e:
        print(f"Error extrayendo {url}: {e}")
        return []

def procesar_bloque(indice, urls):
    print(f"Procesando bloque {indice + 1} con {len(urls)} URLs...")
    resultados = []
    for i, url in enumerate(urls):
        print(f"  [{i+1}/{len(urls)}] {url}")
        detalles = extraer_detalles(url)
        resultados.append({
            "url": url,
            "detalles_ficha": detalles
        })
        time.sleep(0.2)
    archivo = os.path.join(CARPETA_BLOQUES, f"bloque_{indice + 1}.json")
    with open(archivo, "w", encoding="utf-8") as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)
    print(f"Bloque {indice + 1} guardado en {archivo}")

def dividir_en_bloques(lista_urls, tamano):
    return [lista_urls[i:i + tamano] for i in range(0, len(lista_urls), tamano)]

def main():
    print("Leyendo CSV de URLs...")
    df = pd.read_csv("anuncios_unificados1.csv")
    if "url" not in df.columns:
        raise ValueError("El CSV no contiene la columna 'url'.")

    urls = df["url"].dropna().unique().tolist()
    print(f"Total URLs únicas: {len(urls)}")

    bloques_urls = dividir_en_bloques(urls, TAMANO_BLOQUE)
    print(f"Dividido en {len(bloques_urls)} bloques de tamaño {TAMANO_BLOQUE}")

    estado = cargar_estado()

    for i, bloque in enumerate(bloques_urls):
        if estado.get(str(i)) is True:
            print(f"Bloque {i + 1} ya procesado. Saltando.")
            continue
        procesar_bloque(i, bloque)
        estado[str(i)] = True
        guardar_estado(estado)
        print(f"Estado actualizado. Bloque {i + 1} completado.\n")

if __name__ == "__main__":
    main()
